In [32]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import json
from datetime import datetime
import logging
from pathlib import Path 
import concurrent.futures
import tqdm
import yfinance as yf
import pandas as pd
import os
from os.path import join
from datetime import datetime
from pathlib import Path 

In [33]:
# Read the csv from stagging
path = '../../data/intermediate_02/financial-sheets.csv'
ticker_file_all = pd.read_csv(path)
ticker = 'ITRN'

In [34]:
ticker_file = ticker_file_all[ticker_file_all['ticker'] == ticker]

In [35]:
from yahoofinancials import YahooFinancials
from datetime import datetime
import pandas as pd


# Get today's date
end_date = datetime.today().strftime('%Y-%m-%d')

# Set a very early start date
start_date = '1900-01-01'

# Create YahooFinancials object
yahoo_financials = YahooFinancials(ticker)

# Retrieve monthly historical prices
data = yahoo_financials.get_historical_price_data(start_date, end_date, 'monthly')

# Extract the historical prices
historical_prices = data[ticker]['prices']

# Create a DataFrame from the historical prices
hist = pd.DataFrame(historical_prices)

# Capitalize first letter of each column
hist.columns = hist.columns.str.title()

hist.drop(columns='Date', inplace=True)
# Rename the "Formatted_Date" column to "Date"
hist = hist.rename(columns={'Formatted_Date': 'Date'})

# Print the updated DataFrame
hist['Date'] = pd.to_datetime(hist['Date'])
hist['year'] = hist['Date'].dt.year
hist.dropna(inplace=True)

hist



,High,Low,Open,Close,Volume,Adjclose,Date,year
0,13.230000,11.320000,13.150000,13.000000,2031300,5.925489,2005-10-01,2005
1,15.400000,12.650000,13.000000,14.540000,2896000,6.627431,2005-11-01,2005
2,17.129999,14.200000,14.390000,15.950000,2252900,7.270121,2005-12-01,2005
3,18.690001,15.600000,16.389999,17.760000,2063600,8.095132,2006-01-01,2006
4,18.200001,14.830000,17.670000,17.000000,1831000,7.748715,2006-02-01,2006
...,...,...,...,...,...,...,...,...
209,23.059999,20.879999,22.459999,21.780001,492000,21.640387,2023-03-01,2023
210,22.250000,20.740000,21.920000,20.799999,357300,20.799999,2023-04-01,2023
211,24.000000,20.350000,20.920000,22.840000,746200,22.840000,2023-05-01,2023
212,25.500000,22.389999,22.730000,23.990000,382200,23.990000,2023-06-01,2023


In [36]:
# Convert 'Date' column in 'hist' dataframe to string format
hist['Date'] = hist['Date'].astype(str)
# Extract the 'YYYY-MM' format from 'Date' column in 'ticker_file' dataframe
ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)
# Filter 'hist' dataframe based on 'YYYY-MM' values from 'ticker_file'
filtered_hist = hist[hist['Date'].str.slice(0, 7).isin(ticker_file['YearMonth'])]

C:\Users\germa\AppData\Local\Temp/ipykernel_12816/3425731416.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)


In [37]:
ticker_file['YearMonth']

26251    2009-12
26252    2010-12
26253    2011-12
26254    2012-12
26255    2013-12
26256    2014-12
26257    2015-12
26258    2016-12
26259    2017-12
26260    2018-12
26261    2019-12
26262    2020-12
26263    2021-12
26264    2022-12
Name: YearMonth, dtype: object

In [38]:
filtered_hist

,High,Low,Open,Close,Volume,Adjclose,Date,year
50,13.680000,12.670000,12.960000,12.820000,291100,6.960516,2009-12-01,2009
62,17.799999,15.520000,15.890000,17.490000,376800,10.421598,2010-12-01,2010
74,14.000000,12.220000,12.680000,13.660000,189600,8.667988,2011-12-01,2011
86,13.580000,12.750000,13.080000,13.570000,382700,9.637797,2012-12-01,2012
98,21.879999,19.520000,19.920000,21.639999,619500,16.110310,2013-12-01,2013
110,23.490000,21.180000,21.629999,22.040001,369400,17.089172,2014-12-01,2014
122,20.600000,17.760000,20.170000,18.930000,310200,15.243419,2015-12-01,2015
134,26.799999,24.570000,25.000000,26.500000,477700,22.207010,2016-12-01,2016
146,35.779999,33.400002,35.599998,34.200001,788300,29.511410,2017-12-01,2017
158,35.169998,29.379999,33.290001,32.060001,688900,28.477018,2018-12-01,2018


In [39]:
# Format to datetime
filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])
hist['Date'] = pd.to_datetime(hist['Date'])
# Find the earliest date in the filtered_hist dataframe
earliest_date = filtered_hist['Date'].min()
# Filter 'hist' dataframe for rows with a date greater than the earliest date
filtered_hist_after_earliest = hist[hist['Date'].dt.year > earliest_date.year]
# Find the earliest date in the filtered_hist dataframe
earliest_date = filtered_hist['Date'].min()
# Filter 'hist' dataframe for rows with a date greater than the earliest date
filtered_hist_after_earliest = hist[hist['Date'].dt.year < earliest_date.year]

C:\Users\germa\AppData\Local\Temp/ipykernel_12816/2001008434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])


In [40]:
filtered_hist_after_earliest

,High,Low,Open,Close,Volume,Adjclose,Date,year
0,13.230000,11.32,13.150000,13.000000,2031300,5.925489,2005-10-01,2005
1,15.400000,12.65,13.000000,14.540000,2896000,6.627431,2005-11-01,2005
2,17.129999,14.20,14.390000,15.950000,2252900,7.270121,2005-12-01,2005
3,18.690001,15.60,16.389999,17.760000,2063600,8.095132,2006-01-01,2006
4,18.200001,14.83,17.670000,17.000000,1831000,7.748715,2006-02-01,2006
5,17.430000,15.07,17.000000,15.130000,1455700,6.896358,2006-03-01,2006
6,17.379999,15.32,15.320000,16.639999,599100,7.662519,2006-04-01,2006
7,17.370001,13.71,16.600000,14.870000,2365400,6.847456,2006-05-01,2006
8,15.500000,13.66,14.830000,14.010000,2380600,6.451434,2006-06-01,2006
9,15.380000,13.11,14.220000,14.430000,1017100,6.644839,2006-07-01,2006


In [41]:
# Filter the resulting dataframe for rows with the same month as the earliest date
appended_hist = filtered_hist_after_earliest[filtered_hist_after_earliest['Date'].dt.month == earliest_date.month]

In [42]:
appended_hist

,High,Low,Open,Close,Volume,Adjclose,Date,year
2,17.129999,14.20,14.39,15.95,2252900,7.270121,2005-12-01,2005
14,16.730000,14.56,15.96,15.05,912300,6.930341,2006-12-01,2006
26,11.200000,9.95,10.29,11.07,859900,5.171218,2007-12-01,2007
38,8.940000,7.15,7.72,7.31,1223100,3.875585,2008-12-01,2008


In [43]:
# Concatenate the filtered_hist dataframe with the appended_hist dataframe
prices_filtered = pd.concat([filtered_hist, appended_hist]).sort_values(by='year').reset_index(drop=True)

In [44]:
prices_filtered

,High,Low,Open,Close,Volume,Adjclose,Date,year
0,17.129999,14.200000,14.390000,15.950000,2252900,7.270121,2005-12-01,2005
1,16.730000,14.560000,15.960000,15.050000,912300,6.930341,2006-12-01,2006
2,11.200000,9.950000,10.290000,11.070000,859900,5.171218,2007-12-01,2007
3,8.940000,7.150000,7.720000,7.310000,1223100,3.875585,2008-12-01,2008
4,13.680000,12.670000,12.960000,12.820000,291100,6.960516,2009-12-01,2009
5,17.799999,15.520000,15.890000,17.490000,376800,10.421598,2010-12-01,2010
6,14.000000,12.220000,12.680000,13.660000,189600,8.667988,2011-12-01,2011
7,13.580000,12.750000,13.080000,13.570000,382700,9.637797,2012-12-01,2012
8,21.879999,19.520000,19.920000,21.639999,619500,16.110310,2013-12-01,2013
9,23.490000,21.180000,21.629999,22.040001,369400,17.089172,2014-12-01,2014


In [45]:
prices_filtered['ticker'] = ticker
prices_filtered['Growth -1'] = (prices_filtered['Close'] - prices_filtered['Close'].shift(periods=1)) /  prices_filtered['Close'].shift(periods=1)
prices_filtered['Growth +1'] = (prices_filtered['Close'].shift(periods=-1) - prices_filtered['Close']) /  prices_filtered['Close']
prices_filtered['Growth +5'] = (prices_filtered['Close'].shift(periods=-5) - prices_filtered['Close']) /  prices_filtered['Close']
prices_filtered['avgGrowth -10'] = prices_filtered['Growth -1'].rolling(min_periods=10, window=10).sum() / 10
prices_filtered['avgGrowth -5'] = prices_filtered['Growth -1'].rolling(min_periods=5, window=5).sum() / 5

# filter_years = pd.to_datetime(ticker_file['Date']).dt.year.tolist()
# histgrouped_filtered = prices_filtered[prices_filtered['year'].isin(filter_years)]
prices_filtered.reset_index(names=['longevity'], inplace=True)

In [46]:
prices_filtered.head(15)

,longevity,High,Low,Open,Close,Volume,Adjclose,Date,year,ticker,Growth -1,Growth +1,Growth +5,avgGrowth -10,avgGrowth -5
0,0,17.129999,14.200000,14.390000,15.950000,2252900,7.270121,2005-12-01,2005,ITRN,NaN,-0.056426,0.096552,NaN,NaN
1,1,16.730000,14.560000,15.960000,15.050000,912300,6.930341,2006-12-01,2006,ITRN,-0.056426,-0.264452,-0.092359,NaN,NaN
2,2,11.200000,9.950000,10.290000,11.070000,859900,5.171218,2007-12-01,2007,ITRN,-0.264452,-0.339657,0.225836,NaN,NaN
3,3,8.940000,7.150000,7.720000,7.310000,1223100,3.875585,2008-12-01,2008,ITRN,-0.339657,0.753762,1.960328,NaN,NaN
4,4,13.680000,12.670000,12.960000,12.820000,291100,6.960516,2009-12-01,2009,ITRN,0.753762,0.364275,0.719189,NaN,NaN
5,5,17.799999,15.520000,15.890000,17.490000,376800,10.421598,2010-12-01,2010,ITRN,0.364275,-0.218982,0.082333,NaN,0.091500
6,6,14.000000,12.220000,12.680000,13.660000,189600,8.667988,2011-12-01,2011,ITRN,-0.218982,-0.006589,0.939971,NaN,0.058989
7,7,13.580000,12.750000,13.080000,13.570000,382700,9.637797,2012-12-01,2012,ITRN,-0.006589,0.594694,1.520265,NaN,0.110562
8,8,21.879999,19.520000,19.920000,21.639999,619500,16.110310,2013-12-01,2013,ITRN,0.594694,0.018484,0.481516,NaN,0.297432
9,9,23.490000,21.180000,21.629999,22.040001,369400,17.089172,2014-12-01,2014,ITRN,0.018484,-0.141107,0.140200,NaN,0.150376


In [47]:
from market_growth_analysis.etl import web_scraping

path = '../../data/intermediate_02/financial-sheets.csv'
ticker_file_all = pd.read_csv(path)

web_scraping.get_price_info('ITRN', ticker_file_all)

c:\Users\germa\AppData\Local\Programs\Python\Python39\lib\site-packages\market_growth_analysis\etl\web_scraping.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)
c:\Users\germa\AppData\Local\Programs\Python\Python39\lib\site-packages\market_growth_analysis\etl\web_scraping.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])


,longevity,High,Low,Open,Close,Volume,Adjclose,Date,year,ticker,Growth -1,Growth +1,Growth +5,avgGrowth -10,avgGrowth -5
0,0,17.129999,14.200000,14.390000,15.950000,2252900,7.270120,2005-12-01,2005,ITRN,NaN,-0.056426,0.096552,NaN,NaN
1,1,16.730000,14.560000,15.960000,15.050000,912300,6.930341,2006-12-01,2006,ITRN,-0.056426,-0.264452,-0.092359,NaN,NaN
2,2,11.200000,9.950000,10.290000,11.070000,859900,5.171217,2007-12-01,2007,ITRN,-0.264452,-0.339657,0.225836,NaN,NaN
3,3,8.940000,7.150000,7.720000,7.310000,1223100,3.875585,2008-12-01,2008,ITRN,-0.339657,0.753762,1.960328,NaN,NaN
4,4,13.680000,12.670000,12.960000,12.820000,291100,6.960517,2009-12-01,2009,ITRN,0.753762,0.364275,0.719189,NaN,NaN
5,5,17.799999,15.520000,15.890000,17.490000,376800,10.421598,2010-12-01,2010,ITRN,0.364275,-0.218982,0.082333,NaN,0.091500
6,6,14.000000,12.220000,12.680000,13.660000,189600,8.667989,2011-12-01,2011,ITRN,-0.218982,-0.006589,0.939971,NaN,0.058989
7,7,13.580000,12.750000,13.080000,13.570000,382700,9.637794,2012-12-01,2012,ITRN,-0.006589,0.594694,1.520265,NaN,0.110562
8,8,21.879999,19.520000,19.920000,21.639999,619500,16.110310,2013-12-01,2013,ITRN,0.594694,0.018484,0.481516,NaN,0.297432
9,9,23.490000,21.180000,21.629999,22.040001,369400,17.089169,2014-12-01,2014,ITRN,0.018484,-0.141107,0.140200,NaN,0.150376
